In [1]:
import time
import ssim
#!pip install pyssim
from PIL import Image
from io import BytesIO
import requests as r

In [628]:
token = ''
owner_id = '' #ID вашей страницы
version = 5.131 #Версия API

In [ ]:
# Необходимые массивы
images = []
warn = []

In [663]:
def get_data(owner_id, version):
    output = {}
    for i in range(10):
        count = 1000
        album_id = 'saved' #Можете указать свой
        method = 'photos.get'
        offset = i * 1000
        parameters= f'owner_id={owner_id}&count={count}&album_id={album_id}&photo_ids=0&offset={offset}'
        get_saved = f'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
        resp = r.get(get_saved).json()
        try:
            for j in range(1000):
                output.update({resp['response']['items'][j]['id']:resp['response']['items'][j]['sizes'][3]['url']})
        except IndexError:
            pass
    return output


data = get_data(owner_id, version)

In [ ]:
len(data)
#Смотрим сколько всего картинок

### Скачиваем картинки, и добавляем их в массив images

In [666]:
for i in range(len(images), len(data)):
    images.append(Image.open(BytesIO(r.get(data[list(data.keys())[i]]).content)))

In [ ]:
len(images)
#Смотрим сколько картинок скачали

### Сравниваем картинки между собой

In [710]:
#В моём случае фото-двойники оказались рядом, поэтому и сделал скрипт для проверки фото с 10 последующими
for i in range(len(data)):
    img1 = images[i]
    try:
        for j in range(i + 1, i + 11):
            img2 = images[j]
            sim = ssim.compute_ssim(img1, img2)
            if sim > 0.6: # Если хотите найти дубликаты, то замените параметр на 1, хотя рекомендую >0.9
                warn.append([list(data.keys())[i] ,  list(data.keys())[j], sim])
    except IndexError:
        pass
    if i % 10 == 0:
        print(f'Выполнено: {i} из {len(data)}. {(i / len(data)) * 100} %')

Выполнено: 9860 из 9946. 99.13533078624572 %
Выполнено: 9870 из 9946. 99.23587371807761 %
Выполнено: 9880 из 9946. 99.33641664990951 %
Выполнено: 9890 из 9946. 99.4369595817414 %
Выполнено: 9900 из 9946. 99.5375025135733 %
Выполнено: 9910 из 9946. 99.63804544540518 %
Выполнено: 9920 из 9946. 99.73858837723708 %
Выполнено: 9930 из 9946. 99.83913130906897 %
Выполнено: 9940 из 9946. 99.93967424090087 %


### Запрос для удаления всех фото-"двойников"

In [ ]:
method = 'photos.delete'
version = 5.131
try:
    for i in range(len(warn)):
        if warn[i][2] > 0.6:
            photo_id = warn[i][0]
            if warn[i][0] == warn[i+1][0]:
                photo_id = warn[i][1]
            parameters= f'owner_id={owner_id}&photo_id={photo_id}'
            delete_saved = f'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
            time.sleep(0.25)
            resp = r.get(delete_saved)
            print(photo_id, resp.text)
except IndexError:
    pass

## Вот и всё, скрипт автоматически удалил все дубликаты, и очень похожие фото